Midterm Corrections
By: Renee Catanach
G20220457

A Robot of 5 arms 

In [8]:
import numpy as np #import package - array management, math functions
import cv2 # we can display images to the screen

def makeTmat(a,b): 
    T = np.eye(3,3)
    T[0,2] = a
    T[1,2] = b
    return T

def makeRmat(deg):
    rad = deg2rad(deg)
    c = np.cos(rad)
    s = np.sin(rad)
    R = np.eye(3,3)
    R[0,0] = c
    R[0,1] = -s
    R[1,0] = s
    R[1,1] = c
    return R

def getLine(x0,y0,x1,y1):
    points=[]
    if abs(x1-x0)>=abs(y1-y0):
        if x0>=x1:
            for x in range(x0,x1-1,-1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x,int(y)))
        else:
            for x in range(x0,x1+1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x,int(y)))
    else:
        if y0>=y1:
            for y in range(y0,y1-1,-1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x),y))
        else:
            for y in range(y0,y1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x),y))
    return points


def drawLine(canvas, x0, y0, x1, y1, color=(255, 255, 255)):
    
    xys = getLine(x0, y0, x1, y1)
    for xy in xys:
        x, y = xy
        canvas[y, x, :] = color
    
    return


def deg2rad(deg):
    rad = deg * np.pi / 180.
    return rad 

# def getRegularNGon(ngon):
#     delta = 360. / ngon
#     points = []
#     for i in range(ngon):
#         degree = i * delta 
#         radian = deg2rad(degree)
#         x = np.cos(radian)
#         y = np.sin(radian)
#         points.append( (x, y, 1) )
#     #
#     points = np.array(points)
#     return points 

def getRect(h,w):  
    points = []
    points.append( (0, 0, 1))
    points.append( (0, w, 1))
    points.append( (h, w, 1))
    points.append( (h, 0, 1))
    points = np.array(points)
    return points

def drawLinePQ(canvas, p, q, color):
    drawLine(canvas, p[0], p[1], q[0], q[1], color)
    return 

def drawPolygon(canvas, pts, color, axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k,0], pts[k,1], 
                        pts[k+1,0], pts[k+1,1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True: 
        center = np.array([0., 0, 0])
        for p in pts:
            center += p 
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color=(255, 128, 128))
    #
    return 

def erasePolygon(canvas, pts, color=(0,0,0), axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k,0], pts[k,1], 
                        pts[k+1,0], pts[k+1,1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True: # center - pts[0]
        center = np.array([0., 0, 0])
        for p in pts:
            center += p 
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color)
    return

def rotatePoints(degree, points):
    R = makeRmat(30) # define how many degrees you would like to rotate
    qT = R @ points.T
    points = qT.T 
    return points 



width, height = 1400, 1000
canvas = np.zeros( (height, width, 3), dtype='uint8')
alpha = 5
beta = 10
teta = 5

while True:
    # 2. regular n-gon
        color = np.random.randint(0, 256, size=3)
        ngon = 4 # np.random.randint(3, 13)
        h = 60
        w = 130
        rect1 = getRect(h, w) # vertices of the N-gon
        rect2 = rect1.copy()  # hard copy
        rect3 = rect1.copy()  # hard copy
        rect4 = rect1.copy()  # hard copy
        rect5 = rect1.copy()

        # Rect 1
        Tc = makeTmat(600,400)
        R1 = makeRmat(-180)
        T1 = makeTmat(w/2,0)
        H = Tc @ R1 @ T1
        rect1 = (H @ rect1.T).T
        rect1 = rect1.astype('int')
        drawPolygon(canvas, rect1, (0, 0, 255), axis=True)

        # Rect 2
        T2 = makeTmat(0,w)
        T3 = makeTmat(h/2,0)
        T4 = makeTmat(-h/2,0)
        R2 = makeRmat(45)
        H2 = H @ T2 @ T3 @ R2 @ T4
        rect2 = (H2 @ rect2.T).T
        rect2 = rect2.astype('int')
        drawPolygon(canvas, rect2, (160, 0, 255))

        # Rect 3
        R2 = makeRmat(35)
        H3 = H2 @ T2 @ T3 @ R2 @ T4
        rect3 = (H3 @ rect3.T).T
        rect3 = rect3.astype('int')
        drawPolygon(canvas, rect3, (10, 80, 255))

        # Rect 4
        R2 = makeRmat(75)
        H4 = H3 @ T2 @ T3 @ R2 @ T4
        rect4 = (H4 @ rect4.T).T
        rect4 = rect4.astype('int')
        drawPolygon(canvas, rect4, (160, 10, 255))

        # Rect 5
        R2 = makeRmat(95)
        H5 = H4 @ T2 @ T3 @ R2 @ T4
        rect5 = (H5 @ rect5.T).T
        rect5 = rect5.astype('int')
        drawPolygon(canvas, rect5, (80, 15, 255))

        cv2.imshow("my window", canvas)
        if cv2.waitKey(20) == 27: break

        erasePolygon(canvas, rect1, axis=True)

        alpha += 3
        beta += 3
        teta += -2


[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 335   1]
[505 3

KeyboardInterrupt: 

Stars twinkling in night sky

In [9]:
import numpy as np
import cv2
import time

def makeT(a, b):
    T = np.eye(3, 3)
    T[0, 2] = a
    T[1, 2] = b
    return T


def makeR(deg):
    rad = deg2rad(deg)
    c = np.cos(rad)
    s = np.sin(rad)
    R = np.eye(3, 3)
    R[0, 0] = c
    R[0, 1] = -s
    R[1, 0] = s
    R[1, 1] = c
    return R


def getLine(x0, y0, x1, y1):
    points = []
    if abs(x1-x0) >= abs(y1-y0):
        if x0 >= x1:
            for x in range(x0, x1-1, -1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x, int(y)))
        else:
            for x in range(x0, x1+1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x, int(y)))
    else:
        if y0 >= y1:
            for y in range(y0, y1-1, -1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x), y))
        else:
            for y in range(y0, y1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x), y))
    return points


def drawLine(canvas, x0, y0, x1, y1, color=(255, 255, 255)):
    if True:
        xys = getLine(x0, y0, x1, y1)
        for xy in xys:
            x, y = xy
            canvas[y, x, :] = color

    
#


def deg2rad(deg):
    rad = deg * np.pi / 180.
    return rad


def getRegularNGon(ngon):
    delta = 360. / ngon
    points = []
    for i in range(ngon):
        degree = i * delta
        radian = deg2rad(degree)
        x = np.cos(radian)
        y = np.sin(radian)
        points.append((x, y, 1))
    #
    points = np.array(points)
    return points


def drawLinePQ(canvas, p, q, color):
    drawLine(canvas, p[0], p[1], q[0], q[1], color)
    return


def drawPolygon(canvas, pts, color, axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k, 0], pts[k, 1],
                 pts[k+1, 0], pts[k+1, 1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True:  # center - pts[0]
        center = np.array([0., 0, 0])
        for p in pts:
            center += p
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color=(255, 128, 128))
    #
    return


def drawStar(canvas, pts, color, axis=False):
    drawLine(canvas, pts[0, 0], pts[0, 1], pts[2, 0], pts[2, 1], color)
    drawLine(canvas, pts[0, 0], pts[0, 1], pts[3, 0], pts[3, 1], color)
    drawLine(canvas, pts[1, 0], pts[1, 1], pts[3, 0], pts[3, 1], color)
    drawLine(canvas, pts[1, 0], pts[1, 1], pts[4, 0], pts[4, 1], color)
    drawLine(canvas, pts[2, 0], pts[2, 1], pts[4, 0], pts[4, 1], color)
    return


def erasePolygon(canvas, pts, color=(0, 0, 0), axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k, 0], pts[k, 1],
                 pts[k+1, 0], pts[k+1, 1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True:  # center - pts[0]
        center = np.array([0., 0, 0])
        for p in pts:
            center += p
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color)
    return


def eraseStar(canvas, pts, color=(0, 0, 0), axis=False):
    drawLine(canvas, pts[0, 0], pts[0, 1], pts[2, 0], pts[2, 1], color)
    drawLine(canvas, pts[0, 0], pts[0, 1], pts[3, 0], pts[3, 1], color)
    drawLine(canvas, pts[1, 0], pts[1, 1], pts[3, 0], pts[3, 1], color)
    drawLine(canvas, pts[1, 0], pts[1, 1], pts[4, 0], pts[4, 1], color)
    drawLine(canvas, pts[2, 0], pts[2, 1], pts[4, 0], pts[4, 1], color)
    return


def rotatePoints(degree, points):
    R = makeR(30)
    qT = R @ points.T
    points = qT.T
    return points


def main():
    width, height = 1400, 1000
    canvas = np.zeros((height, width, 3), dtype='uint8')

    while True:
        # 2. regular n-gon
        star1Pts = getRegularNGon(5)  # vertices of the N-gon
        star2Pts = getRegularNGon(5)  # vertices of the N-gon
        star3Pts = getRegularNGon(5)  # vertices of the N-gon
        star4Pts = getRegularNGon(5)  # vertices of the N-gon

        # Poly
        star1Pts *= 40
        star1Pts[:, 2] /= 40

        star1x = np.random.randint(100, 1200)
        star1y = np.random.randint(100, 700)
        Tc = makeT(star1x, star1y)
        star1Pts = (Tc @ star1Pts.T).T
        star1Pts = star1Pts.astype('int')
        drawStar(canvas, star1Pts, np.random.randint(0, 256, size=3), axis=False)

        star2Pts *= 20
        star2Pts[:, 2] /= 20

        star2x = np.random.randint(100, 1200)
        star2y = np.random.randint(100, 700)
        Tc = makeT(star2x, star2y)
        star2Pts = (Tc @ star2Pts.T).T
        star2Pts = star2Pts.astype('int')
        drawStar(canvas, star2Pts, np.random.randint(0, 256, size=3), axis=False)

        star3Pts *= 10
        star3Pts[:, 2] /= 10

        star3x = np.random.randint(100, 1200)
        star3y = np.random.randint(100, 700)
        Tc = makeT(star3x, star3y)
        star3Pts = (Tc @ star3Pts.T).T
        star3Pts = star3Pts.astype('int')
        drawStar(canvas, star3Pts, np.random.randint(0, 256, size=3), axis=False)

        star4Pts *= 50
        star4Pts[:, 2] /= 50

        star4x = np.random.randint(100, 1200)
        star4y = np.random.randint(100, 700)
        Tc = makeT(star4x, star4y)
        star4Pts = (Tc @ star4Pts.T).T
        star4Pts = star4Pts.astype('int')
        drawStar(canvas, star4Pts, np.random.randint(0, 256, size=3), axis=False)

        cv2.imshow("my window", canvas)
        if cv2.waitKey(20) == 27:
            break

        drawStar(canvas, star1Pts, (0, 0, 0), axis=False)
        drawStar(canvas, star2Pts, (0, 0, 0), axis=False)
        drawStar(canvas, star3Pts, (0, 0, 0), axis=False)
        drawStar(canvas, star4Pts, (0, 0, 0), axis=False)

        time.sleep(0.3)
    #
#


if __name__ == "__main__":  # __
    main()

KeyboardInterrupt: 

solar system with the sun, venus, earth, moon, and a rocket wondering around.

In [10]:
import numpy as np 
import cv2 

def makeT(a,b):
    T = np.eye(3,3)
    T[0,2] = a
    T[1,2] = b
    return T

def makeR(deg):
    rad = deg2rad(deg)
    c = np.cos(rad)
    s = np.sin(rad)
    R = np.eye(3,3)
    R[0,0] = c
    R[0,1] = -s
    R[1,0] = s
    R[1,1] = c
    return R

def getLine(x0,y0,x1,y1):
    points=[]
    if abs(x1-x0)>=abs(y1-y0):
        if x0>=x1:
            for x in range(x0,x1-1,-1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x,int(y)))
        else:
            for x in range(x0,x1+1):
                y = (x-x0)*(y1-y0)/(x1-x0) + y0
                points.append((x,int(y)))
    else:
        if y0>=y1:
            for y in range(y0,y1-1,-1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x),y))
        else:
            for y in range(y0,y1):
                x = (x1-x0)*(y-y0)/(y1-y0) + x0
                points.append((int(x),y))
    return points


def drawLine(canvas, x0, y0, x1, y1, color=(255, 255, 255)):
    if True:
        xys = getLine(x0, y0, x1, y1)
        for xy in xys:
            x, y = xy
            canvas[y, x, :] = color
        

def deg2rad(deg):
    rad = deg * np.pi / 180.
    return rad 

def getRegularNGon(ngon):
    delta = 360. / ngon
    points = []
    for i in range(ngon):
        degree = i * delta 
        radian = deg2rad(degree)
        x = np.cos(radian)
        y = np.sin(radian)
        points.append( (x, y, 1) )
    #
    points = np.array(points)
    return points 

def drawLinePQ(canvas, p, q, color):
    drawLine(canvas, p[0], p[1], q[0], q[1], color)
    return 

def drawPolygon(canvas, pts, color, axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k,0], pts[k,1], 
                        pts[k+1,0], pts[k+1,1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True: # center - pts[0]
        center = np.array([0., 0, 0])
        for p in pts:
            center += p 
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color=(255, 128, 128))
    #
    return 

def drawStar(canvas, pts, color, axis=False):
    drawLine(canvas, pts[0,0], pts[0,1], pts[2,0], pts[2,1], color)
    drawLine(canvas, pts[0,0], pts[0,1], pts[3,0], pts[3,1], color)
    drawLine(canvas, pts[1,0], pts[1,1], pts[3,0], pts[3,1], color)
    drawLine(canvas, pts[1,0], pts[1,1], pts[4,0], pts[4,1], color)
    drawLine(canvas, pts[2,0], pts[2,1], pts[4,0], pts[4,1], color)
    return

def erasePolygon(canvas, pts, color=(0,0,0), axis=False):
    for k in range(pts.shape[0]-1):
        drawLine(canvas, pts[k,0], pts[k,1], 
                        pts[k+1,0], pts[k+1,1], color)
    drawLinePQ(canvas, pts[-1], pts[0], color)

    if axis == True: # center - pts[0]
        center = np.array([0., 0, 0])
        for p in pts:
            center += p 
        center = center / pts.shape[0]
        center = center.astype('int')
        print(center)
        drawLinePQ(canvas, center, pts[0], color)
    return

def eraseStar(canvas, pts, color=(0,0,0), axis=False):
    drawLine(canvas, pts[0,0], pts[0,1], pts[2,0], pts[2,1], color)
    drawLine(canvas, pts[0,0], pts[0,1], pts[3,0], pts[3,1], color)
    drawLine(canvas, pts[1,0], pts[1,1], pts[3,0], pts[3,1], color)
    drawLine(canvas, pts[1,0], pts[1,1], pts[4,0], pts[4,1], color)
    drawLine(canvas, pts[2,0], pts[2,1], pts[4,0], pts[4,1], color)
    return


def rotatePoints(degree, points):
    R = makeR(30)
    qT = R @ points.T
    points = qT.T 
    return points 

def main():
    width, height = 1400, 1000
    canvas = np.zeros( (height, width, 3), dtype='uint8')
    alpha = 5
    beta = 10
    teta = 5

    while True:
        # 2. regular n-gon
        color = np.random.randint(0, 256, size=3)
        ngon = 5 # np.random.randint(3, 13)
        starPts = getRegularNGon(ngon) # vertices of the N-gon
        polyPts = starPts.copy()  # hard copy
        planetPts = starPts.copy()  # hard copy
        rocketPts = starPts.copy()
        moonPts = starPts.copy()

        # Poly
        polyPts *= 80
        polyPts[:, 2] /= 80        

        Tc = makeT(700,400)
        R1 = makeR(alpha)
        R2 = makeR(-alpha)
        polyPts = (Tc @ R1 @ polyPts.T).T
        polyPts = polyPts.astype('int')
        drawPolygon(canvas, polyPts, (0, 0, 255), axis=True)

        # Star
        starPts = starPts * 50  # scaling
        starPts[:,2] /= 50

        T1 = makeT(300,0)
        R3 = makeR(-beta)
        Hstar = Tc @ R1 @ T1 @ R2 @ R3
        starPts = (Hstar @ starPts.T).T
        starPts = starPts.astype('int')
        drawStar(canvas, starPts, color)
        #

        #planetM
        planetPts *= 20
        planetPts[:,2] /= 20

        Tm = makeT(150,0)
        R4 = makeR(teta)
        R5 = makeR(-teta)
        planetPts = (Hstar @ R4 @ Tm @ R5 @ planetPts.T).T
        planetPts = planetPts.astype('int')
        drawPolygon(canvas, planetPts, (255,0,0))

        

        rocketPts *= 10
        rocketPts[:,2] /= 20

        Tm = makeT(500,0)
        R6 = makeR(alpha)
        R7 = makeR(-alpha)
        rocketPts = (Hstar @ R6 @ Tm @ R7 @ rocketPts.T).T
        rocketPts = rocketPts.astype('int')
        drawPolygon(canvas, rocketPts, (255,0,0))


        moonPts *= 35
        moonPts[:,2] /= 35

        Tm = makeT(350,0)
        R8 = makeR(teta)
        R9 = makeR(-teta)
        moonPts = (Hstar @ R8 @ Tm @ R9 @ moonPts.T).T
        moonPts = moonPts.astype('int')
        drawPolygon(canvas, moonPts, (255,0,0))


        cv2.imshow("my window", canvas)
        if cv2.waitKey(20) == 27: break

        erasePolygon(canvas, polyPts, axis=True)
        eraseStar(canvas, starPts)
        erasePolygon(canvas, planetPts)
        erasePolygon(canvas, rocketPts)
        erasePolygon(canvas, moonPts)

        alpha += 3
        beta += 3
        teta += -2
    #
#

if __name__ == "__main__": # __ 
    main()

[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 399   1]
[699 3

IndexError: index 1000 is out of bounds for axis 0 with size 1000